## Importando bibliotecas

In [33]:
import pandas as pd
import numpy as np
import random
import gc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
import sys
sys.path.append(r'/home/jean/projetos/pod-bank/global/')
from util import *
# Definindo a semente
random.seed(16)

/home/jean/projetos/pod-bank/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Leitura dos dados

In [34]:
df_treino_full = pd.read_parquet('../data/abt/abt_train', engine='fastparquet')

In [35]:
df_treino_full.shape

(215257, 552)

In [36]:
# Separando as variáveis de entrada (features) e de saída (target)
X = df_treino_full.drop(columns=["SK_ID_CURR" ,"TARGET"])
y = df_treino_full["TARGET"]

## Primeira seleção de variáveis

In [37]:
df_tratado, colunas_removidas_por_correlacao, metadata = preprocessar_df(
    df=X,
    y_train=y,
    threshold=0.5,
    percentual_preenchimento=70  # só mantém colunas com até 70% de nulos
)


## Nome das colunas selecionadas

In [38]:
nomes_colunas = df_tratado.columns.tolist()

In [39]:
df_tratado.shape

(215257, 159)

## Dataframe Final Selecionado

In [40]:
colunas_finais = nomes_colunas + ["SK_ID_CURR", "TARGET"]
df_treino_full = df_treino_full[colunas_finais]

In [41]:
df_treino_full.shape

(215257, 161)

In [42]:
df_treino_full.head()

,COMMONAREA_MODE,QT_MAX_QT_MAX_DAYS_ENTRY_PAYMENT_U3M_INSTALLMENTS_U12M_PREV_APP,VL_MED_VL_TOT_AMT_PAYMENT_U3M_INSTALLMENTS_U12M_PREV_APP,QT_MAX_QT_MIN_DAYS_ENTRY_PAYMENT_U3M_INSTALLMENTS_U12M_PREV_APP,NONLIVINGAPARTMENTS_AVG,VL_MED_AMT_CREDIT_SUM_LIMIT_U9M_BUREAU,FLOORSMIN_AVG,VL_MED_VL_MED_CNT_INSTALMENT_FUTURE_U6M_CANCELED_POS_CASH_U12M_PREV_APP,QT_MAX_QT_TOT_SK_DPD_U6M_DEMAND_POS_CASH_U12M_PREV_APP,YEARS_BUILD_AVG,...,FLAG_OWN_REALTY,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_OWN_CAR,CODE_GENDER,NAME_CONTRACT_TYPE,SK_ID_CURR,TARGET
0,0.0497,NaN,NaN,NaN,0.0039,NaN,0.3333,NaN,NaN,0.7960,...,N,Married,Higher education,House / apartment,MONDAY,N,F,Cash loans,100003,0
1,0.0029,-16.0,23284.40,-83.0,0.0039,NaN,0.2083,10.0,0.0,0.8436,...,Y,Married,Secondary / secondary special,House / apartment,WEDNESDAY,N,M,Revolving loans,100068,0
2,0.0069,NaN,NaN,NaN,0.0039,NaN,0.2083,NaN,NaN,0.6600,...,N,Civil marriage,Secondary / secondary special,House / apartment,SATURDAY,Y,M,Cash loans,100081,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,Single / not married,Higher education,House / apartment,WEDNESDAY,Y,F,Cash loans,100101,0
4,NaN,-28.0,31290.44,-84.0,NaN,0.0,NaN,22.0,0.0,NaN,...,Y,Married,Secondary / secondary special,House / apartment,WEDNESDAY,Y,M,Cash loans,100110,0


## Salvando lista das primeiras variáveis selecionadas

In [3]:
# Salvar a lista em um arquivo .pkl
import pickle
with open('../artifacts/prd_first_list_features.pkl', 'wb') as f:
    pickle.dump(colunas_finais, f)